In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Not all these imports are necessary, we'll have to go through them and do some cleanup, eliminating the unnecessary ones
import tensorflow as tf
import pandas as pd
import os
from PIL import Image
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Convolution3D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, \
    GlobalMaxPool2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization, LayerNormalization, Reshape

from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv3D
from keras.applications import ResNet50, ResNet101, ResNet152
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras import backend as K

from tqdm import tqdm
from PIL import Image

from skimage.io import imread
from skimage.transform import resize

from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

target = "age"

In [ ]:
#This part is for data augmentation
print('Using data augmentation in real-time.')
    # Preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range 0 to 10 degrees
        
        width_shift_range=0.1,# randomly shift images horizontally (fraction of total width)
        
        height_shift_range=0.1,# randomly shift images vertically (fraction of total height)
    
        horizontal_flip=True,  # randomly flip images
    
        zoom_range=0.1)

In [ ]:
#Just indicating what our target labels are
if target == "gender":
    labels = ['m','f']
    labels_number = [0,1]
else:
    labels = ['(0, 2)','(4, 6)','(8, 12)','(15, 20)','(25, 32)','(38, 43)','(48, 53)','(60, 100)']
    labels_number = [0,1,2,3,4,5,6,7]

In [ ]:
#some required values. Not all are used yet, for example we are still not saving the model
batch_size = 32
if target == "gender":
    number_of_classes = 2
else:
    number_of_classes = 8
epochs = 100 # for testing; use epochs = 100 for training ~30 secs/epoch on CPU
weight_decay = 1e-4
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'
number_of_images = 5
learning_rate = 0.0001
decay = 1e-6

In [ ]:
def create_original_model():
    model = Sequential()
    model.add(Conv2D(32,kernel_size=5,padding='same',activation='relu',input_shape=(64,64,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
        
    model.add(Conv2D(64,kernel_size=5,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
        
    model.add(Conv2D(128,kernel_size=5,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
        
    model.add(Conv2D(256,kernel_size=5,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
        
    model.add(Conv2D(512,kernel_size=5,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
        
    model.add(Flatten())
    if target == "gender":
        model.add(Dense(number_of_classes, activation='sigmoid'))
    else:
        model.add(Dense(number_of_classes, activation='softmax'))
    
    opt = tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay)
        
    # compile the model
    if target == "gender":
        model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
    else:
        model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])        

    return model

In [ ]:
def create_model_levihassner():
    model = Sequential()
    model.add(Conv2D(96, (7,7), (4,4), padding="same", kernel_regularizer=regularizers.l2(weight_decay), input_shape=(64,64,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="same"))
    model.add(BatchNormalization())
    
    model.add(Conv2D(256, (5,5), padding="same", kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="valid"))
    model.add(BatchNormalization())
    
    model.add(Conv2D(384, (3,3), padding="same", kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="valid"))

    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
 
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
 
    model.add(Flatten())
    if target == "gender":
        model.add(Dense(number_of_classes, activation='sigmoid'))
    else:
        model.add(Dense(number_of_classes, activation='softmax'))
    
    # initialise the optimiser
    opt = tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay)
    
    # compile the model
    if target == "gender":
        model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
    else:
        model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
    return model

In [ ]:
#Model definition. This is the first one to use for CPU/GPU
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(64,64,3)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
 
    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
 
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
 
    model.add(Flatten())
    if target == "gender":
        model.add(Dense(number_of_classes, activation='sigmoid'))
    else:
        model.add(Dense(number_of_classes, activation='softmax'))
    # initialise the optimiser
    opt = tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay)
    
    # compile the model
    if target == "gender":
        model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
    else:
        model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
    return model

In [ ]:
def create_model_resnet50():
    img_height, img_width = 64,64
    if target == "gender":
        num_classes = 2
    else:
        num_classes = 8
    base_model = ResNet50(weights='imagenet', include_top = False, input_shape = (64,64,3), classes = num_classes)
    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    if target == "gender":
        x = Dense(num_classes, activation="sigmoid")(x)
    else:
        x = Dense(num_classes, activation="softmax")(x)    
    
    model = Model(base_model.input, x,  name = "base_model")
    
    opt = tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay)
    
    if target == "gender":
        model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
    else:
        model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
    return model

In [ ]:
def create_model_resnet101():
    img_height, img_width = 64,64
    if target == "gender":
        num_classes = 2
    else:
        num_classes = 8
    base_model = ResNet101(weights='imagenet', include_top = False, input_shape = (64,64,3), classes = num_classes)
    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    if target == "gender":
        x = Dense(num_classes, activation="sigmoid")(x)
    else:
        x = Dense(num_classes, activation="softmax")(x)    
    
    model = Model(base_model.input, x,  name = "base_model")
    
    opt = tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay)
    
    if target == "gender":
        model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
    else:
        model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
    return model

In [ ]:
def create_model_resnet152():
    img_height, img_width = 64,64
    if target == "gender":
        num_classes = 2
    else:
        num_classes = 8
    base_model = ResNet152(weights='imagenet', include_top = False, input_shape = (64,64,3), classes = num_classes)
    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    if target == "gender":
        x = Dense(num_classes, activation="sigmoid")(x)
    else:
        x = Dense(num_classes, activation="softmax")(x)    
    
    model = Model(base_model.input, x,  name = "base_model")
    
    opt = tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay)
    
    if target == "gender":
        model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
    else:
        model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
    return model

In [ ]:
model = create_model_levihassner()
#Model structure
model.summary()

In [ ]:
#Some extra information
BATCH_SIZE = 32
IMG_HEIGHT = 64
IMG_WIDTH = 64

#creates the full paths for the images
def create_path(df, base_path):
    df['path'] = df.apply(lambda x: base_path+"aligned/"+x['user_id']+"/landmark_aligned_face.%s.%s"%(x['face_id'], x['original_image']), axis=1)
    return df

#not sure what this is used for but it's creating an extra column in the dataset. I think it's setting the column
#so that it has value 0 if the face if female and 1 if the face is male, however it shouldn't be needed anymore and I think I'll delete it as soon as I'm sure it's useless
def filter_df(df):
    df['f'] = df.gender.apply(lambda x: int(x in ['f', 'm']))
    df = df[df.f == 1]
    return df


#Base directory path for the dataset
base_path = "/kaggle/input/adience/Dataset/"

#This function plots a batch of images with their respective labels to check whether the dataset has been loaded correctly and the labels are right
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(label_batch[n])
        plt.axis('off')
    plt.show()

#Write a short pure-tensorflow function that converts a file path to an (img, label) pair. Note that the label is passed directly so there's no need to compute it:
def process_path(file_path, label):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])


#Function to prepare the dataset for training
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache).prefetch(buffer_size=AUTOTUNE)

        else:
            ds = ds.cache().prefetch(buffer_size=AUTOTUNE)
    
    #ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    ds = ds.batch(BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds


#Function to analyze te speed at which we can read the dataset
import time
default_timeit_steps = 1000

def timeit(ds, steps=default_timeit_steps):
    start = time.time()
    it = iter(ds)
    for i in range(steps):
        batch = next(it)
        if i%10 == 0:
            print('.',end='')
    print()
    end = time.time()
    
    duration = end-start
    print("{} batches: {} s".format(steps, duration))
    print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))


def process_path_pandas(file_path):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img
    
def read_and_resize(filepath, input_shape=(64, 64)):
    im = Image.open((filepath)).convert('RGB')
    im = im.resize(input_shape)
    im_array = np.array(im, dtype="float32")
    return im_array.astype('float32')/255


#All possible indexes for our test fold, we'll be iterating over them to select one fold as the test one
#Es. [1, 2, 3, 4] 0 -> Fold 0 is the test one, the others are the training one
all_indexes = list(range(5))

one_index = list(range(1))
#store here accuracy of model on each possible division of the folds in train/test. Not yet used
accuracies = []
predictions = []
targets = []
losses = []

#tqdm is used for the progress bar
#loop through the five folds choosing in order each one as the test one. In the original code the model would then be
#trained on each of the possible different divisions of train/test folds.
for test_id in tqdm(all_indexes):    
    
    #Get list of ids for the train folds
    train_id = [j for j in all_indexes if j!=test_id]
    print(train_id, test_id)
    
    #Merge the txt files describing each of the training folds
    train_df = pd.concat([pd.read_csv(base_path+"fold_%s_data.txt"%i, sep="\t") for i in train_id])
    test_df = pd.read_csv(base_path+"fold_%s_data.txt"%test_id, sep="\t")

    #Calls filte funciont, still not 100% on why it was done but currently it should be useless. Will most likely delete it soon
    train_df = filter_df(train_df)
    test_df = filter_df(test_df)

    #Check the final shapes of the train and test division
    print(train_df.shape, test_df.shape)

    #Compute the full paths for the images, updating the dataset so that each row has the complete path to the image it's referring to
    train_df = create_path(train_df, base_path=base_path)
    test_df = create_path(test_df, base_path=base_path)

    
    #Removing all rows whose age group is undefined or not inside our possible ranges:
    if target != "gender":
        train_df = train_df[train_df['age'].isin(labels)]        
        test_df = test_df[test_df['age'].isin(labels)]
    
    #Replacing the target variable with integers, instead of the original string  
    if target == "gender":
        train_df = train_df.replace({"gender" : {"f" : 0, "m" : 1}})
        test_df = test_df.replace({"gender" : {"f" : 0, "m" : 1}})
    else:
        train_df = train_df.replace({"age" : {"(0, 2)" : 0,
                                              "(4, 6)" : 1,
                                              "(8, 12)" : 2,
                                              "(15, 20)" : 3,
                                              "(25, 32)" : 4,
                                              "(38, 43)" : 5,
                                              "(48, 53)" : 6,
                                              "(60, 100)" : 7,}})
        test_df = test_df.replace({"age" : {"(0, 2)" : 0,
                                              "(4, 6)" : 1,
                                              "(8, 12)" : 2,
                                              "(15, 20)" : 3,
                                              "(25, 32)" : 4,
                                              "(38, 43)" : 5,
                                              "(48, 53)" : 6,
                                              "(60, 100)" : 7,}})
        
    train_df, val_df = train_test_split(train_df, test_size=0.1)
        
    # Convert labels to one-hot vectors. 
    if target == "gender":
        #The result is 10 -> female, 01->male
        y_train = tf.keras.utils.to_categorical(train_df.gender.values, number_of_classes)
        y_val = tf.keras.utils.to_categorical(val_df.gender.values, number_of_classes)
        y_test = tf.keras.utils.to_categorical(test_df.gender.values, number_of_classes)
    else:
        y_train = tf.keras.utils.to_categorical(train_df.age.values, number_of_classes)
        y_val = tf.keras.utils.to_categorical(val_df.age.values, number_of_classes)
        y_test = tf.keras.utils.to_categorical(test_df.age.values, number_of_classes)

        
    kf = KFold(n_splits = 5, shuffle = False)

    
    model = create_model_resnet152()

    x_train=train_df['path'].apply(read_and_resize).to_numpy()
    x_train_true= np.stack(x_train, axis=0 )
    
    x_val=val_df['path'].apply(read_and_resize).to_numpy()
    x_val_true= np.stack(x_val, axis=0 )
    
    x_test=test_df['path'].apply(read_and_resize).to_numpy()
    x_test_true= np.stack(x_test, axis=0 )
    
    #for train_index, test_index in kf.split(train_df):
        #y_train_kfold, y_test_kfold = y_train[train_index], y_train[test_index]
        #train_kfold = train_df.iloc[train_index]
        #test_kfold =  train_df.iloc[test_index]

        
    
        #Create dataset with the image paths and the target values for both train and test
        #train_list_ds = tf.data.Dataset.from_tensor_slices((train_kfold.path.values, y_train_kfold))

        #val_list_ds = tf.data.Dataset.from_tensor_slices((val_df.path.values, y_val))
    
        #test_list_ds_kfold = tf.data.Dataset.from_tensor_slices((test_kfold.path.values, y_test_kfold))
    
        #Use Dataset.map to follow the image's path and load the actual image data in its place
        #Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
        #train_labeled_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

        #val_labeled_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    
        #test_labeled_ds_kfold = test_list_ds_kfold.map(process_path, num_parallel_calls=AUTOTUNE)

        # This function is commented because it's not needed for training. It prints the shape and label of the first image in the dataset
        #for image, label in train_labeled_ds.take(1):
        #    print("Image shape: ", image.numpy().shape)
        #    print("Label: ", label.numpy())

    
        #train_ds = prepare_for_training(train_labeled_ds)
        #val_ds = prepare_for_training(val_labeled_ds)
        #test_ds_kfold = prepare_for_training(test_labeled_ds_kfold)
    
    
    
        #Function to determine the speed of access to the datasets. Be careful that it loads the entire dataset into memory, so might not fit.
        #timeit(train_ds)
        #timeit(test_ds)
    
        #This part prints a series of images with their labels, to check whether the data has been loaded correclty
        #image_batch, label_batch = next(iter(train_ds))
        #show_batch(image_batch.numpy(), label_batch.numpy())
    


        # set things up to halt training if the accuracy  has stopped increasing
        #callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
        # Fit the model on the batches generated by datagen.flow().
        #model.fit(train_ds, epochs=epochs, batch_size=BATCH_SIZE, callbacks=[callback])
        
        #scores = model.evaluate(test_ds_kfold, verbose=1)
        #print('Validation Test loss:', scores[0])
        #print('Validation Test accuracy:', scores[1])
    
    #train_list_ds = tf.data.Dataset.from_tensor_slices((train_df.path.values, y_train))
    
    #train_labeled_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    
    #train_ds = prepare_for_training(train_labeled_ds)
    
    #test_list_ds = tf.data.Dataset.from_tensor_slices((test_df.path.values, y_test))
    
    #test_labeled_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    
    #test_ds = prepare_for_training(test_labeled_ds)
    
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
        
    model.fit(datagen.flow(x_train_true, y_train, batch_size=batch_size), epochs=epochs, batch_size=BATCH_SIZE, callbacks=[callback], validation_data = (x_val_true, y_val))
    #model.fit(train_ds, epochs=epochs, batch_size=BATCH_SIZE, callbacks=[callback])
    # Save model and weights
    
    # Evaluate  our trained model.
    scores = model.evaluate(x=x_test_true, y=y_test, verbose=1)
    #scores = model.evaluate(test_ds, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    
    y_predict = model.predict(x=x_test_true)
    predictions.append(y_predict)
    targets.append(y_test)
    losses.append(scores[0])
    accuracies.append(scores[1])
    
print("Mean acc : %s (%s) " %  (np.mean(accuracies), np.std(accuracies)))
print("Mean losses : %s (%s) " %  (np.mean(losses), np.std(losses)))

In [ ]:
one_off_accuracies = []
confusion_matrixes = []
for i in range(5):
    y_predict = predictions[i]
    y_test = targets[i]
    y_test_argmax = y_test.argmax(axis=1)
    y_predict_argmax = y_predict.argmax(axis=1)
    matrix = confusion_matrix(y_test_argmax, y_predict_argmax, normalize="true")
    confusion_matrixes.append(matrix)
    
    y_result = y_test_argmax-y_predict_argmax
    one_off_accuracy = 0
    for i in y_result:
        if abs(i) <= 1:
            one_off_accuracy = one_off_accuracy+1
        
    one_off_accuracies.append((one_off_accuracy*100)/len(y_test))
    
for j in range(5):
    print("Confusion matrix: ")
    print(confusion_matrixes[j])
    print("One-off accuracy: ")
    print(one_off_accuracies[j])
print("Mean one-off acc : %s (%s) " %  (np.mean(one_off_accuracies), np.std(one_off_accuracies)))    
print("Mean acc : %s (%s) " %  (np.mean(accuracies), np.std(accuracies)))
print("Mean losses : %s (%s) " %  (np.mean(losses), np.std(losses)))

In [ ]:
    # BUILD CONVOLUTIONAL NEURAL NETWORKS
    nets = 5
    model = [0] *nets

    for j in range(5):
        model[j] = Sequential()
        model[j].add(Conv2D(24,kernel_size=5,padding='same',activation='relu',input_shape=(64,64,3)))
        model[j].add(MaxPooling2D())
        if j>0:
            model[j].add(Conv2D(48,kernel_size=5,padding='same',activation='relu'))
            model[j].add(MaxPooling2D())
        if j>1:
            model[j].add(Conv2D(64,kernel_size=5,padding='same',activation='relu'))
            model[j].add(MaxPooling2D(padding='same'))
        if j>2:
            model[j].add(Dropout(0.2))
            model[j].add(Conv2D(128,kernel_size=5,padding='same',activation='relu'))
            model[j].add(MaxPooling2D(padding='same'))
        if j>3:
            model[j].add(Conv2D(256,kernel_size=5,padding='same',activation='relu'))
            model[j].add(MaxPooling2D(padding='same'))
            model[j].add(Dropout(0.2))
        if j>4:
            model[j].add(Conv2D(512,kernel_size=5,padding='same',activation='relu'))
            model[j].add(MaxPooling2D(padding='same'))
        
        model[j].add(Flatten())
        model[j].add(Dense(number_of_classes, activation='relu'))
        opt = tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay)
        model[j].compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])